# AI Embeddings in PostgreSQL (Pgvector) | Python Code

### Install PostgreSQL

* **[Install PostgreSQL](https://www.w3schools.com/postgresql/postgresql_install.php)**


### Install PGVector

#### 1. **Linux/Mac**

Compile and install the extension (supports Postgres 13+)

```shell
cd /tmp
git clone --branch v0.8.1 https://github.com/pgvector/pgvector.git
cd pgvector
make
sudo make install # may need sudo
```

**Common Errors**

* **make: x86_64-conda-linux-gnu-cc: No such file or directory**
    * If you have conda environment active then deactivate it. This error will disappear. 
* **fatal error: postgres.h: No such file or directory**
    * On Linux, you need to install **"postgresql-server-dev-16"** along with **postgresql** & **postgresql-contrib**. The **postgresql-server-dev-16** has **posgres.h** file needed for compilation of extensions.

#### 2. **Windows**

Ensure [C++ support in Visual Studio](https://learn.microsoft.com/en-us/cpp/build/building-on-the-command-line?view=msvc-170#download-and-install-the-tools) is installed and run x64 Native Tools Command Prompt for VS [version] **as administrator**. Then use nmake to build

```shell
set "PGROOT=C:\Program Files\PostgreSQL\18"
cd %TEMP%
git clone --branch v0.8.1 https://github.com/pgvector/pgvector.git
cd pgvector
nmake /F Makefile.win
nmake /F Makefile.win install
```

Please make a **NOTE** to start command prompt as **administrator** (by right clicking on command prompt icon) otherwise installation could fail.

Some Helper Tutorials for windows installation.

* **[Install Pgvector on Windows](https://mehmetakar.dev/install-pgvector-on-windows/)**
* **[PGVector Installation on Windows](https://www.mindfiretechnology.com/blog/archive/installing-pgvector-in-preparation-for-retrieval-augmented-generation/)**
* **[PGVector Error Solution Stackoverflow](https://stackoverflow.com/questions/76678804/unable-to-install-pgvector-extension-for-postgresql-windows)**

In [12]:
import os

import psycopg2  ## pip install psycopg2-binary
from dotenv import load_dotenv, find_dotenv  ## pip install python-dotenv

import pandas as pd
from openai import OpenAI

load_dotenv(find_dotenv())

True

In [2]:
def create_connection():
    connection = psycopg2.connect(
        host="localhost",  ## URL of Posgresql if you are using AWS, Google Cloud, etc. It should be public.
        database="postgres", ## User your own database if you have created.
        user=os.environ["POSTGRESQL_USER"], ## PosgreSQL User (default postgres)
        password=os.environ["POSTGRESQL_PASS"], ## Password
        #port="5432" ## Optional
    )
    
    connection.autocommit=True
    
    cursor = connection.cursor()

    return connection, cursor

## 1. Enable PGVector Extension

In [3]:
sql = "CREATE EXTENSION IF NOT EXISTS vector;"

connection, cursor = create_connection()

try:
    cursor.execute(sql)
    print("Success")
except Exception as e:
    print(e)
finally:
    cursor.close()
    connection.close()

Success


**Common Error**

```
extension "vector" is not available
DETAIL:  Could not open extension control file "/usr/share/postgresql/16/extension/vector.control": No such file or directory.
HINT:  The extension must first be installed on the system where PostgreSQL is running
```

If you are getting above error it means that PGVector is not installed properly or postgresql is not able to detect it.

## 2. Verify Extension

In [4]:
sql = "SELECT * FROM pg_extension WHERE extname = 'vector';"

connection, cursor = create_connection()

try:
    cursor.execute(sql)
    print(cursor.fetchall())
except Exception as e:
    print(e)
finally:
    cursor.close()
    connection.close()

[(16388, 'vector', 10, 2200, True, '0.8.1', None, None)]


In [5]:
sql = "SELECT * FROM pg_type WHERE typname = 'vector';"

connection, cursor = create_connection()

try:
    cursor.execute(sql)
    print(cursor.fetchall())
except Exception as e:
    print(e)
finally:
    cursor.close()
    connection.close()

[(16389, 'vector', 2200, 10, -1, False, 'b', 'U', False, True, ',', 0, '-', 0, 16395, 'vector_in', 'vector_out', 'vector_recv', 'vector_send', 'vector_typmod_in', '-', '-', 'i', 'e', False, 0, -1, 0, 0, None, None, None)]


## 3. Store Embeddings in DB

In [6]:
movies_df = pd.read_csv("movies.csv")

movies_df.head()

,id,title,description
0,1,Interstellar,A team of explorers travels through a wormhole...
1,2,Inception,A skilled thief enters people’s dreams to stea...
2,3,The Dark Knight,"Batman faces the Joker, a criminal mastermind ..."
3,4,The Matrix,A hacker discovers the world is a simulated re...
4,5,The Social Network,"The story of Facebook’s creation, exploring am..."


### 3.1 Generate Embeddings

In [27]:
client = OpenAI(
    base_url='http://localhost:11434/v1', ## OpenAI URL 
    api_key="",  ## os.environ["OPENAI_API_KEY"]
)

In [33]:
def get_embedding(text):
    response = client.embeddings.create(
        model="all-minilm",  ## bge-m3, bge-large, embeddinggemma, qwen3-embedding, open AI embedding model, etc.
        input=text
    )
    return response.data[0].embedding

embed = get_embedding("test text")

print(f"Embeddings Length: {len(embed)}")

Embeddings Length: 384


In [34]:
movies_df['embedding'] = movies_df['description'].apply(get_embedding)

movies_df.head()

,id,title,description,embedding
0,1,Interstellar,A team of explorers travels through a wormhole...,"[0.006353421, -0.0052625164, -0.010968455, 0.0..."
1,2,Inception,A skilled thief enters people’s dreams to stea...,"[-0.014985947, 0.050707836, -0.019572502, 0.00..."
2,3,The Dark Knight,"Batman faces the Joker, a criminal mastermind ...","[0.0017600332, 0.041685313, -0.06517475, -0.01..."
3,4,The Matrix,A hacker discovers the world is a simulated re...,"[-0.05623959, 0.023475314, -0.091736026, -0.04..."
4,5,The Social Network,"The story of Facebook’s creation, exploring am...","[-0.01666224, 0.029015947, -0.047585856, -0.01..."


### 3.2 Create Table with Embeddings Column

In [35]:
sql = """
CREATE TABLE IF NOT EXISTS movies (
    id SERIAL PRIMARY KEY,
    title TEXT,
    description TEXT,
    embedding VECTOR(384)  -- use 384 for all-minilm
);
"""

connection, cursor = create_connection()

try:
    cursor.execute(sql)
except Exception as e:
    print(e)
finally:
    cursor.close()
    connection.close()

### 3.3 Insert Embeddings into DB

In [38]:
connection, cursor = create_connection()

try:
    for title, desc, embed in movies_df[["title", "description", "embedding"]].values:
        cursor.execute("INSERT INTO movies (title, description, embedding) VALUES (%s, %s, %s)", (title, desc, embed))
except Exception as e:
    print(e)
finally:
    cursor.close()
    connection.close()

## 4. Query Embeddings (Cosine Similarity)

In [41]:
query_text = "A story about astronauts surviving in space"

# Generate embedding for query
query_embedding = get_embedding(query_text)

In [46]:
connection, cursor = create_connection()

sql = """
SELECT title, description
FROM movies
ORDER BY embedding <=> %s
LIMIT 5;
"""

try:
    cursor.execute(sql, (str(query_embedding),)) ## Pass embeddings as String
    results = cursor.fetchall()
    display(pd.DataFrame(results, columns=["title", "description"]))
except Exception as e:
    print(e)
finally:
    cursor.close()
    connection.close()

,title,description
0,Gravity,Two astronauts struggle to survive after their...
1,The Martian,An astronaut stranded on Mars uses science and...
2,Interstellar,A team of explorers travels through a wormhole...
3,Avatar,Humans colonize Pandora to mine resources. A m...
4,Up,An elderly man and a boy embark on a flying ad...


In [59]:
query_text = "Movies about the wizard who lost his nose and got angry"

# Generate embedding for query
query_embedding = get_embedding(query_text)

In [60]:
connection, cursor = create_connection()

sql = """
SELECT title, description
FROM movies
ORDER BY embedding <=> %s
LIMIT 5;
"""

try:
    cursor.execute(sql, (str(query_embedding),))  ## Pass embeddings as String
    results = cursor.fetchall()
    display(pd.DataFrame(results, columns=["title", "description"]))
except Exception as e:
    print(e)
finally:
    cursor.close()
    connection.close()

,title,description
0,Harry Potter and the Goblet of Fire,Harry competes in a magical tournament. Strate...
1,Harry Potter and the Half-Blood Prince,Secrets and romance challenge Harry and friend...
2,Harry Potter and the Philosopher's Stone,A young wizard begins magical education. Frien...
3,Harry Potter and the Order of the Phoenix,"Harry faces an oppressive regime. Leadership, ..."
4,Harry Potter and the Prisoner of Azkaban,Harry faces new threats and learns more about ...


In [61]:
query_text = "Movies about the ring & hobbits"

# Generate embedding for query
query_embedding = get_embedding(query_text)

In [62]:
connection, cursor = create_connection()

sql = """
SELECT title, description
FROM movies
ORDER BY embedding <=> %s
LIMIT 5;
"""

try:
    cursor.execute(sql, (str(query_embedding),)) ## Pass embeddings as String
    results = cursor.fetchall()
    display(pd.DataFrame(results, columns=["title", "description"]))
except Exception as e:
    print(e)
finally:
    cursor.close()
    connection.close()

,title,description
0,The Lord of the Rings: Fellowship of the Ring,A hobbit embarks on a quest to destroy a power...
1,Hobbit: An Unexpected Journey,Bilbo Baggins joins dwarves on a quest to recl...
2,Hobbit: The Desolation of Smaug,Bilbo confronts the dragon Smaug. He grows in ...
3,Tangled,A princess with magical hair escapes captivity...
4,Shrek 2,Shrek continues adventures with friends and fa...


## 5. Distances

**NOTE** - Please make a NOTE that symbol **<=>** return cosine distance [0, 2] and not cosine similarity. So if you want cosine similarity then you have to do **(1- cosine_distance) [1- (embedding <=> '[vector values here]')]**. Cosine distance of 0 means most similar (cosine simlarity ~= 1) and values near 2 means most dissimilar (cosine similarity ~= -1).

1. **Euclidean Distance (vector_l2_ops)**

```sql
SELECT title
FROM movies
ORDER BY embedding <-> '[vector values here]'
LIMIT 5;
```

3. **Cosine Distance (vector_cosine_ops)**

```sql
SELECT title
FROM movies
ORDER BY embedding <=> '[vector values here]'
LIMIT 5;
```

5. **Inner Product (vector_ip_ops)**

```sql
SELECT title
FROM movies
ORDER BY embedding <#> '[vector values here]'
LIMIT 5;
```

| Metric          | Operator | Use Case                         | Notes                          |
| ----------------| -------- | -------------------------------- | ------------------------------ |
| Euclidean       | `<->`    | Continuous data                  | Sensitive to magnitude         |
| Cosine Distance | `<=>`    | Text embeddings, semantic search | Ignores magnitude, angle-based |
| Inner Product   | `<#>`    | Recommendations                  | Larger = more similar          |




## 6. Indexes

### 6.1. ivfflat (Inverted File + Flat Vectors)

* **How it works:** Partitions vectors into clusters (lists). During search, only relevant clusters are scanned.
* **Use case:** Large datasets (thousands to millions of embeddings).
* **Pros:** Much faster search than sequential scan.
* **Cons:** Slightly approximate results; more accurate with more lists.

In [ ]:
connection, cursor = create_connection()

sql = """
CREATE INDEX ON movies
USING ivfflat (embedding vector_cosine_ops) -- SELECT distances (cosine distance, euclidean, or innr product) based on your usage.
WITH (lists = 100);
"""

try:
    cur.execute(sql)
except Exception as e:
    print(e)
finally:
    cursor.close()
    connection.close()

### 6.2. hnsw (Hierarchical Navigable Small World Graph)

* **How it works:** Creates a graph connecting similar vectors. Search traverses the graph to find nearest neighbors.
* **Use case:** Very large datasets; highly accurate nearest-neighbor search.
* **Pros:** Fast, accurate, great for high-dimensional embeddings.
* **Cons:** More memory usage; index building is heavier.

In [ ]:
connection, cursor = create_connection()

sql = """
CREATE INDEX ON movies
USING hnsw (embedding vector_cosine_ops); -- SELECT distances (cosine distance, euclidean, or innr product) based on your usage.
"""

try:
    cur.execute(sql)
except Exception as e:
    print(e)
finally:
    cursor.close()
    connection.close()

### 6.3. brin (Block Range Index)

* **How it works:** Lightweight index storing min/max values of each page.
* **Use case:** Small datasets or when approximate search is acceptable.
* **Pros:** Very low storage overhead.
* **Cons:** Not optimized for exact similarity search.

In [ ]:
connection, cursor = create_connection()

sql = """
CREATE INDEX ON movies USING brin (embedding);
"""

try:
    cur.execute(sql)
except Exception as e:
    print(e)
finally:
    cursor.close()
    connection.close()

#### Key Points

* Indexes speed up similarity search, especially for large datasets.
* **ivfflat** is common for beginners because it’s simple and fast.
* For small datasets , you don’t need an index — sequential search is fine.


| Index Type  | How it Works                                                             | Pros                                             | Cons                                           | When to Use                               |
| ----------- | ------------------------------------------------------------------------ | ------------------------------------------------ | ---------------------------------------------- | ----------------------------------------- |
| **ivfflat** | Partitions vectors into clusters; searches only relevant clusters        | Fast search, simple to use                       | Approximate results, more lists → slower build | Large datasets, beginner-friendly         |
| **hnsw**    | Builds a graph connecting similar vectors; traverses graph for neighbors | Fast & accurate, great for high-dimensional data | Higher memory use, heavier build               | Very large datasets, high accuracy needed |
| **brin**    | Stores min/max of vector blocks                                          | Very low storage, lightweight                    | Not optimized for similarity                   | Small datasets or rough approximation     |


## Summary

In this tutorial, we learned how to use **PostgreSQL as a vector database** for AI embeddings using the PGVector extension. Starting from a small movie dataset, we covered:

* **Installing PGVector** and enabling it in PostgreSQL.
* **Creating a table with a vector column** to store embeddings.
* **Generating embeddings** from movie descriptions using OpenAI/Ollama.
* **Inserting embeddings** into PostgreSQL.
* **Performing similarity search** to find movies semantically close to a query.

We also introduced the concept of **distance metrics (cosine, Euclidean, inner product)** and briefly touched on **index types (ivfflat, hnsw, brin)** to speed up search.

By the end, you can store, query, and experiment with embeddings directly in PostgreSQL — without needing a separate vector database. This lays the foundation for building AI-powered search, recommendation, or RAG systems using tools you already know.